In [356]:
import re
from selenium import webdriver
import time
import os
import pandas as pd
import numpy as np
import random
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# 我使用selenium+firefox drive 进行模拟点击爬取，为防止被反爬我一般将操作时间间隔设为3秒。
# 目前登录自动化会存在问题，会有滑动验证存在需要手动操作。

In [369]:
url = 'https://www.tianyancha.com'
driver = webdriver.Firefox()
driver.get(url)
time.sleep(2)
driver.refresh()

In [261]:
# 输入用户名和密码
username=13166007797
password="chy13579"

In [371]:
# 会遇到滑动验证，如果没有成功点击尝试多次运行本段
driver.find_element_by_xpath("//*[@onclick='header.loginLink(event)']").click()
driver.find_element_by_xpath("//*[text()='密码登录']").click()
driver.find_element_by_xpath("//*[@id='mobile']").send_keys(username)
driver.find_element_by_xpath("//*[@id='password']").send_keys(password)
driver.find_element_by_xpath("//*[text()='登录']").click()

In [372]:
# 使用搜索框进行搜索
word = "晨兴资本" # 搜索内容
driver.find_element_by_xpath(".//*[@id='home-main-search']").send_keys(word)  
driver.find_element_by_xpath(".//*[@class='input-group-btn btn -xl' and contains(@onclick,'main')]").click()  

In [373]:
# 获取企业专页
base = driver.find_element_by_xpath("//div[@class='position-rel']/a")
url = base.get_attribute("href")
print(url)

https://www.tianyancha.com/organize/b52133021


In [412]:
# 进入页面， 搜寻表格
driver.get(url)
table_public = driver.find_element_by_xpath("//span[@class = 'text' and contains(text(), '公开投资事件')]/../../div[@class='block-data']/div/table")

In [413]:
# 获得列名
col = table_public.find_element_by_xpath("thead").find_elements_by_tag_name("th")
col = [i.text for i in col]
print(col)

['序号', '产品名称', '所属公司', '参与轮次', '投资时间', '投资金额', '产品介绍']


In [414]:
# 获得数据, 针对特定单元在html中实际为table进行特判制定不同爬取方法
pattern_1 = "tbody/tr[$]/td[?]"
pattern_2 = "tbody/tr[$]/td[?]//td[2]"
pattern_3 = "tbody/tr[$]/td[7]/div/div"
indi = {}
rows = []
flag = 1
while flag:
    # 翻页后需要重新定位，否则会有报错
    table_public = driver.find_element_by_xpath("//span[@class = 'text' and contains(text(), '公开投资事件')]/../../div[@class='block-data']/div/table")
    total_td = len(table_public.find_element_by_xpath("tbody").find_elements_by_tag_name("td"))
    unit_td = len(table_public.find_element_by_xpath("tbody/tr[1]").find_elements_by_tag_name("td"))
    num_row = int(total_td/unit_td)
    for j in range(1, num_row+1):
        cache = []
        for i in range(1, 8):
            if i in [2, 3]:
                table_public = driver.find_element_by_xpath("//span[@class = 'text' and contains(text(), '公开投资事件')]/../../div[@class='block-data']/div/table")
                iput = pattern_2.replace("?", str(i))
                iput = iput.replace("$", str(j))
                content = table_public.find_element_by_xpath(iput).text
                cache.append(content)
                if i ==2:
                    link = table_public.find_element_by_xpath(iput+"/a").get_attribute("href")
                    indi[content] = link
            elif i == 7:
                iput = pattern_3.replace("$", str(j))
                cache.append(table_public.find_element_by_xpath(iput).text)
            else:
                iput = pattern_1.replace("?", str(i))
                iput = iput.replace("$", str(j))
                cache.append(table_public.find_element_by_xpath(iput).text)
        rows.append(cache)
    try: # 如果没有翻页则改变flag停止循环
        next_tag = driver.find_element_by_xpath("//*[@id='_investment']//*[@class='num -next']")
        driver.execute_script('arguments[0].click()',next_tag) # 消除覆盖层影响
        time.sleep(3)
    except Exception:
        flag = 0

In [379]:
invest = np.array(rows)
invest = pd.DataFrame(invest, columns=col)

In [380]:
invest.set_index(["序号"], inplace=True)
invest.head()

,产品名称,所属公司,参与轮次,投资时间,投资金额,产品介绍
序号,,,,,,
1,CREAMS,杭州匠人网络科技有限公司,B+轮,2020-04-02,5000万人民币,匠人网络于2016年4月正式推出商业不动产资产管理体系——CREAMS。经过近两年的大力推广...
2,绿洲VR,上海唯二网络科技有限公司,A轮,2020-02-20,千万级美元,唯二科技主要运营的产品叫“绿洲Oasis”，灵感来自于小说《玩家一号》（被斯皮尔伯格拍成电影...
3,深尚科技,北京深尚科技有限公司,天使轮,2019-12-27,600万美元,深尚科技是一家时尚产业解决方案提供商，致力于用AI提供设计相关的时尚产业解决方案。目前深尚科...
4,ONES,深圳复临科技有限公司,B轮,2019-12-24,千万级美元,ONES是一个一站式研发项目管理平台，它的目标客户是以软件研发为核心、人数超过100人的中大...
5,超参数科技,超参数科技（深圳）有限公司,A轮,2019-12-13,千万级美元,超参数科技（深圳）有限公司是一家专注于游戏AI探索的初创公司，主攻机器学习、强化学习、大系统...


In [339]:
invest.to_excel("investment.xls")

## 历史融资进程

In [418]:
# 定义投资者和新闻来源搜寻函数
def get_investors(row, col):
    table_public = driver.find_element_by_xpath("//table[@class = 'table -rongzi -sort']")
    res = []
    pattern = "tbody/tr[{}]/td[{}]".format(str(row), str(col))
    l = table_public.find_element_by_xpath(pattern).find_elements_by_tag_name("a")
    for i in l:
        res.append(i.text)
    return res

In [451]:
# 搜索数据并保存为一个DataFrame， 新增列“公司名”
def get_history(company):
    url = indi[company]
    driver.get(url)
    table_public = driver.find_element_by_xpath("//table[@class = 'table -rongzi -sort']")
    col = table_public.find_element_by_xpath("thead").find_elements_by_tag_name("th")
    col = [i.text for i in col]
    pattern_1 = "tbody/tr[$]/td[?]"
    pattern_2 = "tbody/tr[$]/td[?]//td[2]"
    pattern_3 = "tbody/tr[$]/td[7]/div/div"
    rows = []
    flag = 1
    count = 0
    table_public = driver.find_element_by_xpath("//table[@class = 'table -rongzi -sort']")
    total_td = len(table_public.find_element_by_xpath("tbody").find_elements_by_tag_name("td"))
    unit_td = len(table_public.find_element_by_xpath("tbody/tr[1]").find_elements_by_tag_name("td"))
    num_row = int(total_td/unit_td)
    for j in range(1, num_row+1):
        cache = []
        for i in range(1, 9):
            table_public = driver.find_element_by_xpath("//table[@class = 'table -rongzi -sort']")
            if i in [7, 8]:
                content = get_investors(j, i)
                cache.append(content)
            else:
                iput = pattern_1.replace("?", str(i))
                iput = iput.replace("$", str(j))
                cache.append(table_public.find_element_by_xpath(iput).text)
        rows.append(cache)
    history = np.array(rows)
    history = pd.DataFrame(history, columns=col)
    history.set_index(["序号"], inplace=True)
    history["公司"] = company
    return history

In [457]:
# 合并DataFrame
frames = []
for company in indi:
    frame = get_history(company)
    frames.append(frame)
    time.sleep(3)

In [463]:
history = pd.concat(frames)

In [465]:
history = history.reset_index().drop('序号',axis=1)

In [467]:
history.to_excel("history.xls")